# Testes da rotina que gera os vetores de entrada com base em uma série temporal

## 1. Importando as bibliotecas necessárias

### Bibliotecas obrigatórias

In [1]:
import numpy as np

### Bibliotecas não-obrigatórias

In [2]:
import sys 
sys.path.insert(0, '../../scripts')
import timeseries as ts

## 2. Definindo a função (debug)

Não é necessário executar essa linha se você estiver importando o arquivo com a classe.

In [62]:
class SerieTemporal:

    def __init__(self, dados, K, L):
        """
        Descrição:
        ----------
        Construtor da classe 'SerieTemporal'

        Parâmetros:
        -----------
        dados: np.array
            Conjunto de valores da série temporal
        K: int
            O número de entradas utilizado para a predição
        L: int
            O passo de predição     
        """

        if (K <= 0):
            raise ValueError("O hiperparâmetro 'K' deve ser um inteiro positivo!")

        if not type(dados) is np.ndarray:
            raise TypeError("Os dados devem ser um array do numpy!")

        self._dados = dados
        self._K = K
        self._L = L
        self._matriz_entrada = np.array([])
        self._matriz_saida = np.array([])

    def dividir_treino_teste(self, tam_teste):
        """
        Descrição:
        ----------
        Função para selecionar os 'n' primeiros dados das matrizes para o treinamento.

        Parâmetros:
        -----------
        tam_teste: float
            Proporção de dados que iremos separar para o teste. Deve ser entre 0.0 e 1.0.
        """
        
        if ((tam_teste < 0.0) | (tam_teste > 1.0)):
            raise ValueError("A proporção dos dados de teste deve ser entre 0.0 e 1.0!")
            
        tam_treino = 1.0 - tam_teste
        n_dados = len(self._matriz_saida)

        return (self._matriz_entrada[:int(tam_treino*n_dados)], self._matriz_entrada[int(n_dados*(1 - tam_teste)):],
                self._matriz_saida[:int(tam_treino*n_dados)], self._matriz_saida[int(n_dados*(1 - tam_teste)):])

    def criar_vetores(self, indice):
        """
        Descrição:
        ----------
        Função para criar os vetores de entrada e saída para as etapas de treinamento e teste para uma série temporal.
    
        Parâmetros:
        -----------
        indice: int
            A posicao a partir da qual se deseja prever algum valor
        """

        K, L = self._K, self._L
    
        # checa se a partir da posição atual podemos criar um vetor de amostras dado um K
        if (indice < (K-1)):
            raise ValueError("(indice + 1) =", (indice + 1) , "deve ser maior ou igual a (K - 1) =", (K - 1) , "!")
        
        # checa se o valor que queremos prever (que vai ser armazenado na matriz de saida), está dentro da série temporal
        if ((indice+L) > (len(self._dados)-1)):
            raise ValueError("O passo de predição (L =", L,") somado com o índice atual (indice = ", indice,") não deve estourar o número de dados na série temporal!")
    
        vetor_entrada = np.array(self._dados[(indice-(K-1)):(indice+1)])
        vetor_entrada = np.insert(vetor_entrada, 0, 1) # insere o elemento x^0 no vetor
        vetor_saida = np.array(self._dados[indice+L])
    
        return vetor_entrada, vetor_saida

    def criar_matrizes(self):
        """
        Descrição:
        ----------
        Função para criar as matrizes com os vetores de entrada e saída para as etapas de treinamento e teste.
    
        Parâmetros:
        -----------
        Nenhum
        """
    
        K, L = self._K, self._L
        num_dados = len(self._dados)
    
        for indice in range((K-1), (num_dados-L)):
            vetor_entrada, vetor_saida = self.criar_vetores(indice)
        
            if (len(self._matriz_entrada) == 0):
                self._matriz_entrada = vetor_entrada
            else:
                self._matriz_entrada = np.vstack((self._matriz_entrada, vetor_entrada))
                
            if (len(self._matriz_saida) == 0):
                self._matriz_saida = np.array([vetor_saida])
            else:
                self._matriz_saida = np.vstack((self._matriz_saida, vetor_saida))

## 3. Testando

In [3]:
serie_temporal = ts.SerieTemporal(dados=np.array(range(0, 100)), K=4, L=1)

In [4]:
serie_temporal.criar_matrizes()

In [5]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste=0.35)

In [6]:
print("Matriz de entrada do conjunto de treinamento:\n", X_treino, "\n")
print("Matriz de saída do conjunto de treinamento:\n", y_treino, "\n")
print("Matriz de entrada do conjunto de teste:\n", X_teste, "\n")
print("Matriz de saída do conjunto de teste:\n", y_teste)

Matriz de entrada do conjunto de treinamento:
 [[ 1  0  1  2  3]
 [ 1  1  2  3  4]
 [ 1  2  3  4  5]
 [ 1  3  4  5  6]
 [ 1  4  5  6  7]
 [ 1  5  6  7  8]
 [ 1  6  7  8  9]
 [ 1  7  8  9 10]
 [ 1  8  9 10 11]
 [ 1  9 10 11 12]
 [ 1 10 11 12 13]
 [ 1 11 12 13 14]
 [ 1 12 13 14 15]
 [ 1 13 14 15 16]
 [ 1 14 15 16 17]
 [ 1 15 16 17 18]
 [ 1 16 17 18 19]
 [ 1 17 18 19 20]
 [ 1 18 19 20 21]
 [ 1 19 20 21 22]
 [ 1 20 21 22 23]
 [ 1 21 22 23 24]
 [ 1 22 23 24 25]
 [ 1 23 24 25 26]
 [ 1 24 25 26 27]
 [ 1 25 26 27 28]
 [ 1 26 27 28 29]
 [ 1 27 28 29 30]
 [ 1 28 29 30 31]
 [ 1 29 30 31 32]
 [ 1 30 31 32 33]
 [ 1 31 32 33 34]
 [ 1 32 33 34 35]
 [ 1 33 34 35 36]
 [ 1 34 35 36 37]
 [ 1 35 36 37 38]
 [ 1 36 37 38 39]
 [ 1 37 38 39 40]
 [ 1 38 39 40 41]
 [ 1 39 40 41 42]
 [ 1 40 41 42 43]
 [ 1 41 42 43 44]
 [ 1 42 43 44 45]
 [ 1 43 44 45 46]
 [ 1 44 45 46 47]
 [ 1 45 46 47 48]
 [ 1 46 47 48 49]
 [ 1 47 48 49 50]
 [ 1 48 49 50 51]
 [ 1 49 50 51 52]
 [ 1 50 51 52 53]
 [ 1 51 52 53 54]
 [ 1 52 53 54 55]